In [1]:
%pylab inline
#Import matplotlib & numpy namespace

Populating the interactive namespace from numpy and matplotlib


In [18]:
# NE PAS EXECUTER 
#VOICI COMMENT ONE ETE CODEES LES FEATURES A PREDIRE contenant les combinaisons de carte (du dealer et des joueurs) et de DECISIONS  (du joueur) 
#LE BUT EST DE PREDIRE LE GAIN POUR CHACUNES DE CES CONFIGURATIONS POUR ELABORER UNE STRATEGIE
#Le fichier retourné est dans la partie_ML>ALL_FILES>STATES_ALL>MAT_STATESALL.csv
"""
OPTIONS_CH=['HIT','STAND','DOUBLE']
CARDS=['2','3','4','5','6','7','8','9','10','A']
OPTIONS_CO=[str(i) for i in range(4,22)]+['A'+'-'+i for i in CARDS if i!='A']
OPTIONS_DEAL=[str(i) for i in range(2,10)]+['A']
values_9_10_11=['A-8','A-9','A-10','9','10','11']
STATES=[['DEAL'],['LAST_CO'],['LAST_CH']]

for i in OPTIONS_DEAL:
    for j in range(len(OPTIONS_CO)):
        if OPTIONS_CO[j] in values_9_10_11:
        
            
            for h in range(3):
                STATES[0].append(i)
                STATES[1].append(OPTIONS_CO[j])
            for opt in['STAND','HIT','DOUBLE']:
                STATES[2].append(opt)
            
            
        else:
            for h in range(2):
                STATES[0].append(i)
                STATES[1].append(OPTIONS_CO[j])
            for opt in['STAND','HIT']:
                STATES[2].append(opt)
            

MAT_STATES=np.array(STATES)
MAT_STATES=MAT_STATES.transpose()

    
nom='/Users/williamlambert/Desktop/INSTALL PYGAME/BLACKJACK 0.0/PARTIE_ML/ALL_FILES/STATES_ALL/MAT_STATESALL.csv'
np.savetxt(nom, MAT_STATES, delimiter=",",fmt='%s')
"""


In [2]:
#Voici comment les données ont été traitées : et placées dans un unique fichier pour appliquer le RandomForest 
#correctement

#IMPORTATION DES DONNEES , FUSION DES DONNEES ET TRAITEMENT POUR LE RANDOM FOREST

#LES DONNEES RANDOM CORRESPONDENT A 24300 PARTIES JOUEES DE MANIERE ALEATOIRE A MISE CONSTANTE=25 $
"""
import csv,numpy as np
NAME_MAT1='ALL_FILES/DATA_RANDOM.csv'
r = csv.reader(open(NAME_MAT1)) 
lines = list(r)
lines.pop(0)
for i in range(len(lines[3])):
        if lines[3][i]=="-50":
            lines[3][i]=1
        elif lines[3][i]=="-25":
            lines[3][i]=2
        elif lines[3][i]=="0":
            lines[3][i]=3
        elif lines[3][i]=="25":
            lines[3][i]=4
        elif lines[3][i]=="50":
            lines[3][i]=5
MAT=np.array(lines)

#DONNEES COLLECTEES EN JOUANT DES PARTIES ALEATOIRES
ALL_OTHER_DATA=['ALL_FILES/DATA_RANDOM'+str(i)+'.csv' for i in range(2,9)]

#POUR CHAQUE FICHIER ON ENLEVE SA PREMIERE LIGNE CONTENANT LE NOM DES VARIABLES ET ON L'AJOUTE AU PREMIER
#ON RECODE LE GAIN DE 1 A 5 POUR POUVOIR CALCULER L'ERREURS DES PREDICTIONS DANS LA SUITE 
for DATA_NAME in ALL_OTHER_DATA:
    r = csv.reader(open(DATA_NAME)) 
    lines = list(r)
    lines.pop(0)
    
    for i in range(len(lines)):
        lines[i].pop(0)

    for i in range(len(lines[3])):
        if lines[3][i]=="-50":
            lines[3][i]=1
        elif lines[3][i]=="-25":
            lines[3][i]=2
        elif lines[3][i]=="0":
            lines[3][i]=3
        elif lines[3][i]=="25":
            lines[3][i]=4
        elif lines[3][i]=="50":
            lines[3][i]=5
    MAT1=np.array(lines)
    MAT=np.concatenate((MAT,MAT1),axis=1)
    
    
MAT=MAT.transpose()


#RETOURNE CE FICHIER CSV TRAITEE 
writer = csv.writer(open('ALL_FILES/DATA_RANDOM_TRAITEE.csv','w'))
writer.writerows(MAT)

"""


In [3]:
# Pandas est utilisé pour la manipulation de données
import pandas as pd
# Lis les données collectées et affiche les 5 premières lignes
features = pd.read_csv('ALL_FILES/DATA_RANDOM_TRAITEE.csv')
features.head(5)

#Lis les données a predire et affiche les 5 premieres lignes
features_STRAT= pd.read_csv('ALL_FILES/STATES_ALL/MAT_STATESALL.csv')
features_STRAT.head(5)






/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:53: UserWarning: [Errno 2] No such file or directory: 'arch': 'arch'
  stacklevel=stacklevel + 1)
/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:53: UserWarning: [Errno 2] No such file or directory: 'machine': 'machine'
  stacklevel=stacklevel + 1)
/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory: 'sysctl': 'sysctl'
  stacklevel=stacklevel + 1):


,DEAL,LAST_CO,LAST_CH
0,2,4,STAND
1,2,4,HIT
2,2,5,STAND
3,2,5,HIT
4,2,6,STAND


In [4]:
#STATISTIQUES DESCRIPTIVES SUR LES DONNEES
#(UNIQUEMENT POUR LES VARIABLES QUANTITATIVES : ici : GAIN)

features.describe()
#A NOTER LA MOYENNE DE GAIN est de 2.79 soit entre -25$ (codé en 2) et -0$ (codé en 3)

,GAIN
count,24375.000000
mean,2.789662
std,1.099740
min,1.000000
25%,2.000000
50%,2.000000
75%,4.000000
max,5.000000


In [5]:
# TRAITEMENT DES DONNEES : TRANSFORMATION DES DONNEES EN "DUMMIES": 
# pd.get_dummies DE TRAITER TOUTES LES VARIABLES QUANTITATIVES QUI PRENNENT UN NOMBRE DE MODALITES FINIES N, EN 
#LES TRANSFORMANT EN N VARIABLES DONT LE NOM DEVIENT VAR_MODALITE_i et qui comme prend comme valeur 0 ou 1 

#ON CREER LES DUMMIES ET ON AFFICHE UNE PARTIE DES NOUVELLES DONNEES OBTENUES
features = pd.get_dummies(features)
features.iloc[:,5:].head(5)

#IDEM POUR LES DONNEES A PREDIRE
features_STRAT=pd.get_dummies(features_STRAT)
features_STRAT.iloc[:,5:].head(5)




,DEAL_7,DEAL_8,DEAL_9,DEAL_A,LAST_CO_10,LAST_CO_11,LAST_CO_12,LAST_CO_13,LAST_CO_14,LAST_CO_15,...,LAST_CO_A-3,LAST_CO_A-4,LAST_CO_A-5,LAST_CO_A-6,LAST_CO_A-7,LAST_CO_A-8,LAST_CO_A-9,LAST_CH_DOUBLE,LAST_CH_HIT,LAST_CH_STAND
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
#A LIRE : SI L 'ALGO RENVOIT UNE ERREUR A LA CELLULE CI DESSOUS IL SUFFIT DE TOUT REEXECUTER (ERREUR BIZARRE)

In [7]:
# Import Numpy
import numpy as np

# LES LABELS SONT LES VALEURS A PREDIRE
labels = np.array(features['GAIN'])

# ON RETIRE LE LABEL DES FEATURES (=LES DONNEES PERMETTANT DE PREDIRE LE GAIN)
# AXIS 1 veut dire colonnes
features= features.drop('GAIN', axis = 1)
feature_list = list(features.columns)

#IDEM POUR LES DONNEES A PREDIRE (ICI PAS DE GAIN A RETIRER DES DONNEES CAR ON A PAS SES VALEURS)
feature_strat_list=list(features_STRAT.columns)

# CONVERSION EN MATRICE NUMPY
features = np.array(features)
features_STRAT=np.array(features_STRAT)


In [8]:
# UTILISATION DE SCIKIT LEARN POUR SPLITTER LES DONNEES EN ECHANTILLON D'APRENTISSAGE ET ECHANTILLON DE TEST
#LE RANDOM FOREST PERMET DE SELECTIONNER CES DONNEES PAR UNE SELECTION BOOTSTRAP POUR NE PAS FAVORISER UNE PARTIE
#DES DONNEES
#SI ON SOUHAITE PRENDRE TOUTES LES DONNEES ON PEUX FAIRE BOOTSTRAP=FALSE

#IMPORT° DU MODULE SCIKIT LEARN
from sklearn.model_selection import train_test_split

# SPLIT DES DONNEES 
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

#PAS DE DONNEES D'APPRENTISSAGE POUR LES DONNEES A PREDIRE
test_features_STRAT=features_STRAT

In [9]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)


Training Features Shape: (18281, 39)
Training Labels Shape: (18281,)
Testing Features Shape: (6094, 39)
Testing Labels Shape: (6094,)


In [10]:
#IMPORT° DU MODELE RANDOM FOREST QU'ON UTILISE
from sklearn.ensemble import RandomForestRegressor

# INITIALISE LE MODEL AVEC 1000 ARBRES DE DECISIONS
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, bootstrap=False)

# ENTRAINE LE MODEL SUR L'ECHANTILLON D'APPRENTISSAGE
rf.fit(train_features, train_labels);


In [11]:
#PREDICTIONS DU RANDOM FOREST SUR LES DONNEES DE TEST (A UTILISER POUR CALCULER LA PRECISION DU MODEL)
predictions = rf.predict(test_features)
 
#ICI NOS PREDICTIONS POUR ELABORER  LA STRATEGIE
predictions_STRAT=rf.predict(test_features_STRAT)

# CALCUL DE L'ERREUR MOYENNE EN VALEUR ABSOLUE
errors = abs(predictions - test_labels)

# AFFICHAGE DE l'ERREUR MOYENNE CONVERTIE EN DOLLAR
print('Mean Absolute Error:', round(np.mean(errors)*25, 2), '$')


Mean Absolute Error: 18.72 $


In [12]:
# CALCUL DU POURCENTAGE D'ERREUR MOYEN
mape = 100 * (errors / test_labels)

# CALCUL DE LA PRECISION DES PREDICTIONS
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

# 70% EST ASSES SATISFAISANT POUR 24000 DONNEES ET UN JEU ALEATOIRE COMME LE BLACKJACK 
# CELA PEUX NOUS PERMETTRE D'ELABORER UNE STRATEGIE.

#LA VRAIE QUESTION EST DE SAVOIR SI CETTE PRECISION NOUS PERMET DE PRENDRE DE BONNES DECISIONS
#CELA EST ETUDIE DANS LE FICHIER EXCEL STATES_ALL/STRAT PROJET ML 

Accuracy: 69.03 %.


In [13]:
#ON EXPORTE NOS PREDICTIONS DANS UN CSV POUR ELABORER NOTRE STRATEGIE ML

LIST_STRAT=list(predictions_STRAT)
MAT_STRAT=np.array(LIST_STRAT)
np.savetxt("PREDICTIONS_RANDOM_FOREST.csv", MAT_STRAT, delimiter=" ",fmt='%s')

In [14]:
# IMPORT DES MODULES UTILISES POUR LA VISUALISATION
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from sklearn.tree import export_graphviz
import pydot

# SELECTION D'UN ARBRE DANS LA FORET
tree = rf.estimators_[5]

# EXPORTE L'ARBRE DANS UNE FICHIER DOT
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# UTILISE LE FICHIER DOT POUR CREER UN GRAPH
(graph, ) = pydot.graph_from_dot_file('tree.dot')

#ECRIT LE GRAPH DANS UNE IMAGE PNG
graph.write_png('tree.png')




In [15]:
# CREATION D'UN ARBRE MINIATURE DE PROFONDEUR 3 POUR FACILITER LA COMPREHENSION
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# EXTRACTION DU PETIT ARBRE
tree_small = rf_small.estimators_[5]

# SAUVEGARDE LE PETIT ARBRE
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');